In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import csv
import unicodedata
import os

/home/vlad.cristescu/miniconda3/envs/proiect_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-31 15:50:06.039927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def wordopt(text):
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)
    
    text = text.replace('\n', ' ')

    text = text.encode('ascii', 'ignore').decode('ascii')
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
df = pd.read_csv("../../datasets/Combined_Corpus/All.csv")

## Vizualizare articole

In [ ]:
import textwrap

index = 29790                
 
text = df["Statement"].iloc[index]

wrapped = textwrap.fill(text, width=100)
print(wrapped)

How are you? What about this weather weâ€™ve been having. How are the kids? I hear No 42 are digging
out a basement. I donâ€™t know where they get the money. Did you hear what Peter said about Paul?
What Paul said about Paula? We really should do something about the dog mess fouling the street. Do
you want to see the new kitchen? Do you find this sort of thing fascinating â€“ or do you only
pretend? Such topics constitute the essence of small talk, smeared like spermicide over social
intercourse lest the embryo of a real conversation be fertilised. Not that thereâ€™s anything wrong
with small talk per se. Pondering the meaning of life or the meaning of meaning at the school gates
would be unbearable. But enough is enough. Among those who flatter themselves as being educated,
curious and thoughtful â€“ like, I imagine, most Guardian readers â€“ serious talk is becoming a
form of bad manners. I donâ€™t go to dinner parties much now, but when I do, small talk seems to be
the main course w

In [10]:
clean_text = wordopt(text)
wrapped = textwrap.fill(clean_text, width=100)
print(wrapped)

How are you? What about this weather weve been having. How are the kids? I hear No 42 are digging
out a basement. I dont know where they get the money. Did you hear what Peter said about Paul? What
Paul said about Paula? We really should do something about the dog mess fouling the street. Do you
want to see the new kitchen? Do you find this sort of thing fascinating or do you only pretend? Such
topics constitute the essence of small talk, smeared like spermicide over social intercourse lest
the embryo of a real conversation be fertilised. Not that theres anything wrong with small talk per
se. Pondering the meaning of life or the meaning of meaning at the school gates would be unbearable.
But enough is enough. Among those who flatter themselves as being educated, curious and thoughtful
like, I imagine, most Guardian readers serious talk is becoming a form of bad manners. I dont go to
dinner parties much now, but when I do, small talk seems to be the main course with gossip, jokes
and co

In [12]:
not_en = df[df["language"] != "en"]
print(not_en)

                                               Statement  Label  \
518    WISCONSIN: Russ Feingold (D) vs. Ron Johnson (...      0   
841    Peter Schiff Exposes Media Lies About Janet Ye...      0   
1617    Ben Swann Interviews G. Edward GriffinBen Swann       0   
1796   Seit 2009 lebt und arbeitet Nicole Janz in Gro...      1   
2666   El nuevo presidente del PerÃº, Pedro Pablo Kuc...      1   
...                                                  ...    ...   
78978  Bankster Suicides and Bank Run ChatterJames Co...      0   
79029                                     Simone 13 Nov       1   
81135                    saying nasty things about Islam      0   
81278  EXPOSED! U.N. AGENDA: INFILTRATE AMERICA With ...      0   
81379  US BABY FOUND IN MEXICO, MOM FOUND DEAD IN FRI...      0   

       Statement_length  word_count  char_count  avg_word_length language  
518                  57           9          48         5.333333       de  
841                  63           9        

## Articole cu ""

In [ ]:
df['has_quotes'] = df['text'].str.strip().apply(lambda x: x.startswith('"') and x.endswith('"'))

num_articles_with_quotes = df['has_quotes'].sum()
print("Number of articles with quotes at beginning and end:", num_articles_with_quotes)

Number of articles with quotes at beginning and end: 46


In [51]:
df.sample(5)

,Unnamed: 0,title,text,label,language,category,has_quotes
29768,32671,Russia may send more Iskander missiles to Kali...,Russia may send more Iskander tactical missile...,0,en,WORLDPOST,False
46187,52699,Soros-Financed Groups Unite to Sue for Trump W...,Three government watchdog groups have report...,0,en,POLITICS,False
38939,43663,U.S. lawmaker wounded in June shooting dischar...,"U.S. Congressman Steve Scalise, the No. 3 Repu...",0,en,POLITICS,False
53611,62338,Trump Is Trying To Make Facebook Give Him Per...,Lawyers for the Trump administration have serv...,1,en,POLITICS,False
10793,11307,Roy Moore Supporting Republican LOSES It Afte...,If the scandal around the stories of Alabama S...,1,en,POLITICS,False
